In [ ]:
# import required modules
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy
from torch_geometric.utils import degree

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj

from scipy import sparse

In [ ]:
'''
Let's look at the following 3 x 4 interaction matrix 
      items    i1   i2   i3   i4
users     u1    0    0    1   1
          u2    1    0    1   0
          u3    0    1    0   0
          
        
'''

In [ ]:
# torch.from_numpy
# r_mat here is the interaction matrix above
r_mat = np.array([[0, 0, 1, 1], 
                  [1, 0, 1, 0], 
                  [0, 1, 0, 0]])
print(r_mat)

[[0 0 1 1]
 [1 0 1 0]
 [0 1 0 0]]


In [ ]:
r_mat_np_coo = sparse.coo_matrix(r_mat)
print(r_mat_np_coo)

  (0, 2)	1
  (0, 3)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1


In [ ]:
# edge_index is just [row, col] from COO format
r_mat_edge_index =  [r_mat_np_coo.row, r_mat_np_coo.col] 

print(r_mat_edge_index[0])
print(r_mat_edge_index[1])

[0 0 1 1 2]
[2 3 0 2 1]


In [ ]:
print(r_mat_np_coo.todense())

[[0 0 1 1]
 [1 0 1 0]
 [0 1 0 0]]


# You can also do the same with Pytorch 

In [ ]:
r_mat_edge_index = torch.LongTensor(np.array(r_mat_edge_index))
r_mat_edge_index

tensor([[0, 0, 1, 1, 2],
        [2, 3, 0, 2, 1]])

In [ ]:
sparse_r_mat_edge_index = SparseTensor(row=r_mat_edge_index[0], 
                                           col=r_mat_edge_index[1], 
                                           sparse_sizes=(3, 4))

print(sparse_r_mat_edge_index)

SparseTensor(row=tensor([0, 0, 1, 1, 2]),
             col=tensor([2, 3, 0, 2, 1]),
             size=(3, 4), nnz=5, density=41.67%)


In [ ]:
print(sparse_r_mat_edge_index.to_dense())

tensor([[0., 0., 1., 1.],
        [1., 0., 1., 0.],
        [0., 1., 0., 0.]])


In [ ]:
'''
In bipartite graph, interaction matrix is not the same as adjacency matrix, because adjacency matrix
expecteds row index and col index to refer to the same node

perform the conversion between interaction matrix (r_mat) and adjacency matrix (adj_mat)
    ( 0    R )
A = ( R_T  0 )

so if dimension of R is  n_user x n_item
then dimension of A is (n_user+n_item) x (n_user+n_item)

'''

In [ ]:
def convert_r_mat_edge_index_to_adj_mat_edge_index(input_edge_index, row_size, col_size):
    R = torch.zeros((row_size, col_size))
    
    # convert sparse coo forat to dense format to get R
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = 1

    # perform the  r_mat to adj_mat conversion   
    R_transpose = torch.transpose(R, 0, 1)
    adj_mat = torch.zeros((row_size + col_size , row_size + col_size))
    adj_mat[: row_size, row_size :] = R.clone()
    adj_mat[row_size :, : row_size] = R_transpose.clone()
    
    # convert from dense format back to sparse coo format so we can get the edge_index of adj_mat
    adj_mat_coo = adj_mat.to_sparse_coo()
    adj_mat_coo = adj_mat_coo.indices()
    return adj_mat_coo

In [ ]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index, row_size, col_size):
    # create a sparse tensor so we can easily do the to_dense conversion and get a sub matrix to 
    # get R (interaction matrix) and then convert it back to sparse coo format
    sparse_input_edge_index = SparseTensor(row=input_edge_index[0], 
                                           col=input_edge_index[1], 
                                           sparse_sizes=((row_size + col_size), row_size + col_size))
    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[: row_size, row_size :]
    r_mat_edge_index = interact_mat.to_sparse_coo().indices()
    return r_mat_edge_index

In [ ]:
adj_mat_edge_index = convert_r_mat_edge_index_to_adj_mat_edge_index(r_mat_edge_index, row_size=3, col_size=4)
print(adj_mat_edge_index)

tensor([[0, 0, 1, 1, 2, 3, 4, 5, 5, 6],
        [5, 6, 3, 5, 4, 1, 2, 0, 1, 0]])


In [ ]:
adj_mat = SparseTensor(row=adj_mat_edge_index[0], 
                                       col=adj_mat_edge_index[1], 
                                       sparse_sizes=(3+4, 4+3))

print(adj_mat.to_dense())

tensor([[0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]])


In [ ]:
converted_back_to_r_mat_edge_index = convert_adj_mat_edge_index_to_r_mat_edge_index(adj_mat_edge_index, 3, 4)

converted_back_to_r_mat = SparseTensor(row=converted_back_to_r_mat_edge_index[0], 
                                       col=converted_back_to_r_mat_edge_index[1], 
                                       sparse_sizes=(3, 4))

print(converted_back_to_r_mat.to_dense())

tensor([[0., 0., 1., 1.],
        [1., 0., 1., 0.],
        [0., 1., 0., 0.]])
